In [6]:


import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from skimage.filters import threshold_otsu

# Define the U-Net model with ResNet34 backbone
model = smp.Unet(
    encoder_name='resnet152',        # Use ResNet34 as the encoder backbone
    encoder_weights='imagenet',     # Use pretrained weights on ImageNet
    in_channels=3,                  # Input channels, adjust according to your data
    classes=1,                      # Number of output classes, adjust according to your task
)

# Move the model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [45]:
import torch

model.load_state_dict(torch.load('trained_model_all_32.pth'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# # model.eval()  # set the model to evaluation mode


In [46]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    if len(runs) % 2 == 1:  # 짝수 개의 요소로 만들기 위해
        runs = np.append(runs, len(pixels))
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)






In [ ]:
# 이미지 폴더 정의
image_folder='./test_img/'

# 이미지 파일 리스트 불러오기
image_filenames = sorted(os.listdir(image_folder))
print(image_filenames)

['TEST_00000.png', 'TEST_00001.png', 'TEST_00002.png', 'TEST_00003.png', 'TEST_00004.png', 'TEST_00005.png', 'TEST_00006.png', 'TEST_00007.png', 'TEST_00008.png', 'TEST_00009.png', 'TEST_00010.png', 'TEST_00011.png', 'TEST_00012.png', 'TEST_00013.png', 'TEST_00014.png', 'TEST_00015.png', 'TEST_00016.png', 'TEST_00017.png', 'TEST_00018.png', 'TEST_00019.png', 'TEST_00020.png', 'TEST_00021.png', 'TEST_00022.png', 'TEST_00023.png', 'TEST_00024.png', 'TEST_00025.png', 'TEST_00026.png', 'TEST_00027.png', 'TEST_00028.png', 'TEST_00029.png', 'TEST_00030.png', 'TEST_00031.png', 'TEST_00032.png', 'TEST_00033.png', 'TEST_00034.png', 'TEST_00035.png', 'TEST_00036.png', 'TEST_00037.png', 'TEST_00038.png', 'TEST_00039.png', 'TEST_00040.png', 'TEST_00041.png', 'TEST_00042.png', 'TEST_00043.png', 'TEST_00044.png', 'TEST_00045.png', 'TEST_00046.png', 'TEST_00047.png', 'TEST_00048.png', 'TEST_00049.png', 'TEST_00050.png', 'TEST_00051.png', 'TEST_00052.png', 'TEST_00053.png', 'TEST_00054.png', 'TEST_000

In [50]:
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from skimage.filters import threshold_otsu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 디바이스 설정
# 이미지 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),
])

# # 이미지 폴더 정의
# image_folder='./test_img/'

# # 이미지 파일 리스트 불러오기
# image_filenames = sorted(os.listdir(image_folder))

# 결과를 저장할 빈 리스트 생성
results = []
# 이미지 파일 리스트를 반복하면서 처리
for filename in image_filenames:
    # 이미지 파일 경로
    image_path = os.path.join(image_folder, filename)
    # 이미지 불러오기 및 변환
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    # 이미지를 모델 입력 형태에 맞게 변형 (예: 배치 차원 추가)
    image = image.unsqueeze(0)  # shape: [1, C, H, W]
    image = image.to('cuda')
    image1 = Image.open(image_path).convert("RGB")
    # 모델에 이미지 전달 및 결과 저장
    with torch.no_grad():
        output = model(image)
        binary_mask = torch.sigmoid(output).cpu().numpy()
        binary_mask = np.squeeze(binary_mask, axis=0)
        threshold = threshold_otsu(binary_mask)
        binary_mask = (binary_mask > threshold).astype(np.uint8)
        mask_rle = rle_encode(binary_mask[0])
        if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
            results.append(-1)
        else:
            results.append(mask_rle)
        # plt.figure(figsize=(20, 5))
        # plt.subplot(1, 2, 1)
        # plt.imshow(image1)
        # plt.title(filename)

        # plt.subplot(1, 2, 2)
        # plt.imshow(binary_mask[0])
        # plt.title('Predicted Mask')


    print(filename)


    

TEST_00000.png
TEST_00001.png
TEST_00002.png
TEST_00003.png
TEST_00004.png
TEST_00005.png
TEST_00006.png
TEST_00007.png
TEST_00008.png
TEST_00009.png
TEST_00010.png
TEST_00011.png
TEST_00012.png
TEST_00013.png
TEST_00014.png
TEST_00015.png
TEST_00016.png
TEST_00017.png
TEST_00018.png
TEST_00019.png
TEST_00020.png
TEST_00021.png
TEST_00022.png
TEST_00023.png
TEST_00024.png
TEST_00025.png
TEST_00026.png
TEST_00027.png
TEST_00028.png
TEST_00029.png
TEST_00030.png
TEST_00031.png
TEST_00032.png
TEST_00033.png
TEST_00034.png
TEST_00035.png
TEST_00036.png
TEST_00037.png
TEST_00038.png
TEST_00039.png
TEST_00040.png
TEST_00041.png
TEST_00042.png
TEST_00043.png
TEST_00044.png
TEST_00045.png
TEST_00046.png
TEST_00047.png
TEST_00048.png
TEST_00049.png
TEST_00050.png
TEST_00051.png
TEST_00052.png
TEST_00053.png
TEST_00054.png
TEST_00055.png
TEST_00056.png
TEST_00057.png
TEST_00058.png
TEST_00059.png
TEST_00060.png
TEST_00061.png
TEST_00062.png
TEST_00063.png
TEST_00064.png
TEST_00065.png
TEST_00066

In [51]:
import pandas as pd

submit = pd.read_csv('./submission.csv')
submit['mask_rle'] = results

In [52]:
submit.to_csv('./submit_resnet152_32.csv', index=False)

In [27]:
submit.to_csv('./submit_resnet34_33.csv', index=False)

In [ ]:
submit.to_csv('./submit_resnet34_34.csv', index=False)

In [24]:
submit.to_csv('./submit_resnet34_22.csv', index=False)

In [15]:
submit.to_csv('./submit_152to34.csv', index=False)